In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt

In [2]:
dataset = pd.read_csv('news_summary.csv')#,encoding="ISO-8859-1")
dataset.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",Virgin' now corrected to 'Unmarried' in IGIMS'...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [3]:
dataset = dataset.dropna(how='any',axis=0) 
x = dataset['ctext']
y = dataset['text']
headline = dataset['headlines']
del dataset

In [4]:
def preprocess_sentence(w):
    w = w.lower().strip()

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [5]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

In [6]:
x = [preprocess_sentence(w) for w in x]
y = [preprocess_sentence(w) for w in y]
headline = [preprocess_sentence(w) for w in headline]
x_train, inp_lang = tokenize(x)
y_train, targ_lang = tokenize(y)
headline_train , headline_lang = tokenize(headline)
print("Input Tensor Shape :" , x_train.shape , "Target Tensor Shape :" , y_train.shape)
print("Preprocessed Text : \n" , x[10])
print("Tokenized Text : \n" , x_train[10])
print("Headline Text : \n" , headline[10])
print("Headline Tokenized : \n" , headline_train[10])

Input Tensor Shape : (4395, 5791) Target Tensor Shape : (4395, 86)
Preprocessed Text : 
 <start> the food safety and standards authority of india fssai is in the process of creating a network of food banking partners to collect and distribute leftover food from large parties and weddings to the hungry . a notification to create a separate category of food business operators fbos , who will be licensed to deal only with leftover food , has been drafted to ensure the quality of food . ? we are looking at partnering with ngos or organisations that collect , store and distribute surplus food to ensure they maintain certain hygiene and health standards when handling food , ? said pawan agarwal , ceo of fssai . ? tonnes of food is wasted annually . we are looking at creating a mechanism through which food can be collected from restaurants , weddings , large scale parties , ? says pawan agarwal , ? all food , whether it is paid for or distributed free , must meet the country ? s food safety a

In [7]:
class Encoder(tf.keras.Model):
    '''
    The code of encoder is used for text summarization. The model needs an initial_state which is the headline of the article.
    The input is supposed to be padded sequence for initial state as well as for the input. The model cuurently implements
    GRU for the encoder sequence as there is some bug with using lstm with bidirectional. Bidirectional was also tried with 
    GRU but same bug occured.
    
    The initialization is based on docuemnt context vector paper referenced from 
    https://arxiv.org/pdf/1807.08000.pdf paper named 
    Abstractive and Extractive Text Summarization using DocumentContext Vector and Recurrent Neural Networks.
    
    However the logic is modeified. The model takes the preprocessed tokenized padded sequence as input for initials state
    and the then it is passed through the embedded vector and the size of the output vector after embedding is (batch_size , time , embedding size)
    after this the model is averaged about the time axis and gives the output as (batch_size , embedding size).
    This tensor is then passed as input to dence vector and is used to give the output size of (batch_zie , lgur_hidden_units)
    '''
    def __init__(self,units = 16,activation='tanh',recurrent_activation='sigmoid',return_state=True,return_sequence=True , vocab_size=30000 , embedding_dim=30 , batch_size=2):
        super (Encoder , self).__init__()
        self.units = units
        self.activation = activation
        self.recurrent_activation = recurrent_activation
        self.return_state = return_state
        self.return_sequence = return_sequence
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
#         self.lstm = tf.keras.layers.LSTM(units=self.units , activation=self.activation , recurrent_activation=self.activation, return_sequences=self.return_sequence , return_state=self.return_state)
        self.gru = tf.keras.layers.GRU(units=self.units , activation=self.activation , recurrent_activation=self.activation, return_sequences=self.return_sequence , return_state=self.return_state)
#         self.bidirectional = tf.keras.layers.Bidirectional(self.gru)
        self.dense = tf.keras.layers.Dense(units=self.units , input_shape = (self.embedding_dim,))
    
    def call(self,x,headline):
        x = self.embedding(x)
        print(x.shape)
        headline = self.embedding(headline)
        headline = tf.reduce_mean(headline,axis=1)
        headline  = self.dense(headline)
        output , state = self.gru(x,initial_state=headline)
        return output , state

In [8]:
encoder = Encoder()

In [9]:
output , state = encoder(x_train[:2] , headline_train[:2])
print(output)
print(state)

(2, 5791, 30)
tf.Tensor(
[[[ 0.03370849  0.01697699  0.02288595 ...  0.0255639   0.0256319
   -0.03076383]
  [ 0.0209395  -0.01639135 -0.0198533  ...  0.01310598  0.03307112
    0.02388732]
  [-0.00479571 -0.02213357 -0.05248106 ... -0.01754641  0.02041301
   -0.00901647]
  ...
  [ 0.03356769 -0.02665575  0.05065165 ... -0.02117584 -0.02182933
    0.00680007]
  [ 0.03356769 -0.02665575  0.05065165 ... -0.02117584 -0.02182933
    0.00680007]
  [ 0.03356769 -0.02665575  0.05065165 ... -0.02117584 -0.02182933
    0.00680007]]

 [[ 0.0329232   0.01725838  0.02296641 ...  0.02570033  0.02584546
   -0.03112317]
  [-0.00516574 -0.03667758 -0.01311471 ... -0.04633377  0.02854744
    0.04107402]
  [ 0.04356331 -0.02771307  0.01416744 ...  0.02028258 -0.012892
   -0.0253625 ]
  ...
  [ 0.03356769 -0.02665575  0.05065165 ... -0.02117584 -0.02182933
    0.00680007]
  [ 0.03356769 -0.02665575  0.05065165 ... -0.02117584 -0.02182933
    0.00680007]
  [ 0.03356769 -0.02665575  0.05065165 ... -0.02117

In [10]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [11]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(state, output)
print(attention_result)
print(attention_weights)

tf.Tensor(
[[ 0.03167229 -0.02504718  0.04648036  0.01211604 -0.03271927 -0.00174045
   0.00540717 -0.03070412 -0.02240552  0.00062074 -0.01431609 -0.01458084
  -0.03872574 -0.01939478 -0.02021422  0.00645646]
 [ 0.03126151 -0.02471879  0.046278    0.01193519 -0.03219241 -0.00193332
   0.00544128 -0.03033368 -0.02198728  0.00046814 -0.01438568 -0.01461027
  -0.03858323 -0.01931183 -0.02045809  0.00633706]], shape=(2, 16), dtype=float32)
tf.Tensor(
[[[0.00016393]
  [0.00016412]
  [0.00016388]
  ...
  [0.00017305]
  [0.00017305]
  [0.00017305]]

 [[0.00016392]
  [0.0001546 ]
  [0.0001723 ]
  ...
  [0.00017301]
  [0.00017301]
  [0.00017301]]], shape=(2, 5791, 1), dtype=float32)
